In [1]:
!pip install mlflow transformers datasets boto3 

  Using cached mlflow-2.21.3-py3-none-any.whl.metadata (30 kB)
  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached mlflow_skinny-2.21.3-py3-none-any.whl.metadata (31 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached databricks_sdk-0.49.0-py3-none-any.whl.metadata (38 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_sdk-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pydantic-2.11.2-py3-none-any.whl.met

In [1]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np


/Users/adityatandon/.pyenv/versions/3.9.17/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("hpe-ai/medical-cases-classification-tutorial")
dataset

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 1724
    })
    validation: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 370
    })
    test: Dataset({
        features: ['description', 'transcription', 'sample_name', 'medical_specialty', 'keywords'],
        num_rows: 370
    })
})

In [3]:
medical_test = dataset['test']
medical_train = dataset['train']
medical_val = dataset['validation']


In [7]:
train_df = medical_test.to_pandas()
train_df.head()

,description,transcription,sample_name,medical_specialty,keywords
0,"Complete laminectomy, L4. and facetectomy, L3-...","PREOPERATIVE DIAGNOSIS:, Dural tear, postoper...",Laminectomy & Facetectomy,Orthopedic,None
1,"Loculated left effusion, multilobar pneumonia....","REASON FOR CONSULTATION: , Loculated left effu...",Multilobar Pneumonia,Cardiovascular / Pulmonary,"cardiovascular / pulmonary, neck bruits, nasal..."
2,Abnormal echocardiogram findings and followup....,"REASON FOR CONSULTATION: ,Abnormal echocardio...",Abnormal Echocardiogram,Cardiovascular / Pulmonary,None
3,Application of PMT large halo crown and vest. ...,"PREOPERATIVE DIAGNOSES: , Cervical spondylosis...",PMT Halo Crown & Vest,Orthopedic,"orthopedic, cervical spondylosis, anterior cer..."
4,MRI Brain & MRI C-T spine: Multiple hemangiobl...,"CC:, Weakness.,HX:, This 30 y/o RHM was in goo...",MRI Brain and C-T Spine,Neurology,None
